In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('blast.db')
c = conn.cursor()

In [32]:
def get_table_name(word_size):
    return "preprocess_wordsize_" + str(word_size)

In [103]:
nucleotide_num = {'A':0,'C':1,'G':2,'T':3}

def get_word_encoding(word):
    inverted_word = word[::-1]
    n = len(word)
    
    index = 0
    
    for i in range(n):
        index += pow(4,i) * nucleotide_num[inverted_word[i]] 
    
    return index + 1

In [104]:
get_word_encoding('AGCTT')

160

In [3]:
def all_combinations(word_size):
    nucleotides = ('A','C','G','T')
    sol = list()
    
    def all_combinations_rec(size,cur_sol):
    
        # exit condition
        if size == 0:
            sol.append(''.join(cur_sol))
            return
        
        for nuc in nucleotides:
            
            cur_sol.append(nuc)
            all_combinations_rec(size-1,cur_sol)
            
            # backtrack
                       
            cur_sol.pop()
    
    all_combinations_rec(word_size,[])
    
    return sol

In [136]:
def create_word_table(word_size):
    
    # new table name
    new_table_name = get_table_name(word_size)
    
    # Create new table for this word size
    
    create_table_s = "CREATE TABLE {} (id INTEGER PRIMARY KEY,sequence_index int NOT NULL UNIQUE,word_encoding int NOT NULL)".format(new_table_name,str(word_size))
    c.execute(create_table_s)
    
    # Save (commit) the changes
    conn.commit()

In [139]:
create_word_table(5)

In [138]:
# delete a table if it already exists
word_size = 5
new_table_name = get_table_name(word_size)
#c.execute("DROP TABLE {}".format(new_table_name))

In [113]:
# makes a cursor which is like an iterator
c.execute("SELECT * FROM preprocess_wordsize_5")

In [114]:
# you can do this to print them or make them into a list
for row in c.execute("SELECT * FROM preprocess_wordsize_5"):
    print(row)

In [115]:
l = list(c.execute("SELECT * FROM preprocess_wordsize_5"))

In [116]:
c.execute("SELECT * FROM indexed_words_wordsize_5")

In [128]:
def get_indexes_for_word(word):
    word_size = (len(word))
    new_table_name = get_table_name(word_size)
    
    encoding = get_word_encoding(word)
    
    s = "SELECT sequence_index FROM {} where word_encoding = ?".format(new_table_name)
    return c.execute(s,(encoding,))


In [129]:
get_indexes_for_word('AAAAA')

In [140]:
def insert_index_for_word(word,index):
    word_size = (len(word))
    new_table_name = get_table_name(word_size)
    
    encoding = get_word_encoding(word)
    
    s = "INSERT INTO {} (sequence_index,word_encoding) VALUES (?,?)".format(new_table_name)
    c.execute(s,(index,encoding,))
    
    conn.commit()

In [146]:
cursor = get_indexes_for_word('AAAAA')

In [147]:
for row in cursor:
    print(row)